In [6]:
!pip -q install yellowbrick==1.3

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
!pip -q install kneed
!pip -q install factor_analyzer

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer
from yellowbrick.features import PCA as YBPCA
from kneed import KneeLocator
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
#import plotly.express as px

In [44]:
bike = pd.read_csv('../data/bike_clean.csv')
bike.head()

,Unnamed: 0,datetime,date,hour,season,holiday,open,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,1,2017-12-01 00:00:00,2017-12-01,0,Winter,No Holiday,Yes,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,2,2017-12-01 01:00:00,2017-12-01,1,Winter,No Holiday,Yes,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,3,2017-12-01 02:00:00,2017-12-01,2,Winter,No Holiday,Yes,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,4,2017-12-01 03:00:00,2017-12-01,3,Winter,No Holiday,Yes,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,5,2017-12-01 04:00:00,2017-12-01,4,Winter,No Holiday,Yes,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0


In [45]:
# As PCA is a unsupervised learning algorithm, we will drop the Season column
# We will drop datetime and date as well since they are not numeric
season = bike['season']
bike2 = bike
bike2.drop(columns = ['season','datetime','date','holiday','open'],inplace=True)
bike2.head()

,Unnamed: 0,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,1,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,2,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,3,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,4,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,5,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0


In [46]:
# remvove first column
bike3 = bike2.iloc[: , 1:]
bike3.head()

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0


In [47]:
bike3
# there are 8760 rows and 12 attributes
#can try group by season in previous cells and check for each season there how many rows

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,0,254,-5.2,37,2.2,20000,-17.6,0.0,0.0,0
1,1,204,-5.5,38,0.8,20000,-17.6,0.0,0.0,0
2,2,173,-6.0,39,1.0,20000,-17.7,0.0,0.0,0
3,3,107,-6.2,40,0.9,20000,-17.6,0.0,0.0,0
4,4,78,-6.0,36,2.3,20000,-18.6,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
8755,19,1003,4.2,34,2.6,18940,-10.3,0.0,0.0,0
8756,20,764,3.4,37,2.3,20000,-9.9,0.0,0.0,0
8757,21,694,2.6,39,0.3,19680,-9.9,0.0,0.0,0
8758,22,712,2.1,41,1.0,18590,-9.8,0.0,0.0,0


In [48]:
bike3.describe()

# we see each attributes have different ranges

,hour,rent_count,temperature,humidity,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,11.500000,704.602055,12.882922,58.226256,1.724909,14368.257991,4.073813,0.569111,0.148687,0.750685
std,6.922582,644.997468,11.944825,20.362413,1.036300,6082.987120,13.060369,0.868746,1.128193,4.367462
min,0.000000,0.000000,-17.800000,0.000000,0.000000,270.000000,-30.600000,0.000000,0.000000,0.000000
25%,5.750000,191.000000,3.500000,42.000000,0.900000,9400.000000,-4.700000,0.000000,0.000000,0.000000
50%,11.500000,504.500000,13.700000,57.000000,1.500000,16980.000000,5.100000,0.010000,0.000000,0.000000
75%,17.250000,1065.250000,22.500000,74.000000,2.300000,20000.000000,14.800000,0.930000,0.000000,0.000000
max,23.000000,3556.000000,39.400000,98.000000,7.400000,20000.000000,27.200000,3.520000,35.000000,88.000000


CHECK DATA
We want to check whether the data is suitable for PCA. We will use two tests, Bartlett's Sphericity Test and Kaiser Meyer Olkin (KMO) Test to check.

Bartlett's Sphericity Test

In [50]:
_ , p_value = calculate_bartlett_sphericity(bike3)
p_value

0.0

KMO Test

In [52]:
_, kmo_score = calculate_kmo(bike3)
kmo_score 

0.5028116564973184

We obtained a result < 0.05 for Bartlett's test and = 0.503 > 0.5 for KMO's. This means that conducting PCA on our data might be fruitful.

CUMULATIVE VARIANCE AND SCREE PLOT